In [1]:
import json
import os
import matplotlib.pyplot as plt
import numpy as np
import requests
from bs4 import BeautifulSoup
import muspy
import pandas as pd
import torch
from torch.utils.data import DataLoader
from pathlib import PosixPath

In [8]:
def find_local_mscz(raw_path, dpath='/data2/zachary/musescore/data'):
    clean_path = raw_path.split("/")[-1] + '.mscz'
    for dirpath, dirnames, filenames in os.walk(dpath):
        if not dirnames:
            if clean_path in filenames:
                return os.path.join(dirpath, clean_path)

In [9]:
fpath_df = pd.read_csv('/data2/zachary/musescore/mscz-files.csv')
fpath_df.set_index('id', inplace=True)

In [10]:
all_files = []
for dirpath, dirnames, filenames in os.walk('/data2/zachary/musescore/data'):
    if not dirnames:
        all_files += list(map(lambda x: os.path.join(dirpath, x), filenames))

In [11]:
all_files = {x.split("/")[-1]: x for x in all_files}

In [12]:
fpath_df['ref2'] = fpath_df.ref.apply(lambda x: x.split("/")[-1] + '.mscz')
fpath_df['loc_exist'] = fpath_df.ref2.apply(lambda x: x in all_files)
fpath_df = fpath_df[fpath_df['loc_exist'] == True]

In [13]:
fpath_df['ref3'] = fpath_df.ref2.apply(lambda x: all_files[x])

In [2]:
dirpath = '/data2/zachary/musescore/metadata'

In [4]:
compls = []
cntr = 0
for fpath in os.listdir(dirpath):
    if fpath.isnumeric():
        subdir = os.path.join(dirpath, fpath)
        for subpath in os.listdir(subdir):
            with open(subdir + '/' + subpath, 'r') as f:
                try:
                    d = json.load(f)
                    compls.append({'path': subdir + '/' + subpath, 'd': d})
                except:
                    pass
            if cntr % 5000 == 0:
                print(cntr)
            cntr += 1


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000
470000
475000
480000
485000
490000
495000
500000
505000
510000
515000
520000
525000
530000
535000
540000
545000
550000
555000
560000
565000
570000
575000
580000
585000
590000
595000
600000
605000
610000
615000
620000
625000
630000
635000
640000
645000
650000
655000
660000
665000
670000
675000
680000
685000
690000
695000
700000
705000
710000
715000
720000
725000
730

In [15]:
def dur2Num(dur):
    ms = dur.split(":")
    if len(ms) == 1:
        return int(ms[0])
    return int(ms[0]) * 60 + int(ms[1])

In [5]:
import pickle as pk
with open("../data/all_meta.pkl", 'wb') as f:
    pk.dump(compls, f)

In [16]:

pianos = list(filter(lambda x: 'piano' in [y.lower() for y in x['d']['data']['score']['parts_names']] and len(x['d']['data']['score']['parts_names']) == 1 and len(x['d']) and dur2Num(x['d']['data']['score']['duration']) > 20, compls))
pianos = list(filter(lambda x: x['d']['data']['score']['id'] in fpath_df.index, pianos))
nans = list(filter(lambda x: x['d']['data']['score']['complexity'] == 0, pianos))
easy = list(filter(lambda x: x['d']['data']['score']['complexity'] == 1, pianos))
med = list(filter(lambda x: x['d']['data']['score']['complexity'] == 2, pianos))
hard = list(filter(lambda x: x['d']['data']['score']['complexity'] == 3, pianos))

N = min(len(easy), len(med), len(hard))
np.random.seed(42)
easy_eq = np.random.choice(easy, N)
med_eq = np.random.choice(med, N)
hard_eq = np.random.choice(hard, N)
train_ds = np.concatenate([easy_eq[:int(N*0.8)], med_eq[:int(N*0.8)], hard_eq[:int(N*0.8)]])
val_ds = np.concatenate([easy_eq[int(N*0.8):int(N*0.8) + int(N*0.1)], med_eq[int(N*0.8):int(N*0.8) + int(N*0.1)], hard_eq[int(N*0.8):int(N*0.8) + int(N*0.1)]])
test_ds = np.concatenate([easy_eq[int(N*0.8) + int(N*0.1):], med_eq[int(N*0.8) + int(N*0.1):], hard_eq[int(N*0.8) + int(N*0.1):]])

In [19]:
times = []
for piece in easy_eq:
    tcode = piece['d']['data']['score']['duration']
    if len(tcode.split(":")) == 2:
        times.append(int(tcode.split(":")[0]) * 60 + int(tcode.split(":")[1]))
    else:
        times.append(int(tcode.split(":")[0]))
for piece in med_eq:
    tcode = piece['d']['data']['score']['duration']
    if len(tcode.split(":")) == 2:
        times.append(int(tcode.split(":")[0]) * 60 + int(tcode.split(":")[1]))
    else:
        times.append(int(tcode.split(":")[0]))
for piece in hard_eq:
    tcode = piece['d']['data']['score']['duration']
    if len(tcode.split(":")) == 2:
        times.append(int(tcode.split(":")[0]) * 60 + int(tcode.split(":")[1]))
    else:
        times.append(int(tcode.split(":")[0]))

In [20]:
sum(times) / 3600

769.2172222222222

In [21]:
times = []
for piece in train_ds:
    tcode = piece['d']['data']['score']['duration']
    if len(tcode.split(":")) == 2:
        times.append(int(tcode.split(":")[0]) * 60 + int(tcode.split(":")[1]))
    else:
        times.append(int(tcode.split(":")[0]))
print("train time:", sum(times) / 3600)

times = []
for piece in val_ds:
    tcode = piece['d']['data']['score']['duration']
    if len(tcode.split(":")) == 2:
        times.append(int(tcode.split(":")[0]) * 60 + int(tcode.split(":")[1]))
    else:
        times.append(int(tcode.split(":")[0]))
print("val time:", sum(times) / 3600)

times = []
for piece in test_ds:
    tcode = piece['d']['data']['score']['duration']
    if len(tcode.split(":")) == 2:
        times.append(int(tcode.split(":")[0]) * 60 + int(tcode.split(":")[1]))
    else:
        times.append(int(tcode.split(":")[0]))
print("test time:", sum(times) / 3600)

train time: 614.9838888888889
val time: 75.31944444444444
test time: 78.91388888888889


In [22]:
for i, d in enumerate(train_ds):
    raw_path = fpath_df.loc[d['d']['data']['score']['id']]['ref3']
    if type(raw_path) == pd.Series:
        raw_path = raw_path.iloc[0]
    d['mscz_path'] = raw_path
    train_ds[i] = d
for i, d in enumerate(val_ds):
    raw_path = fpath_df.loc[d['d']['data']['score']['id']]['ref3']
    if type(raw_path) == pd.Series:
        raw_path = raw_path.iloc[0]
    d['mscz_path'] = raw_path
    val_ds[i] = d
for i, d in enumerate(test_ds):
    raw_path = fpath_df.loc[d['d']['data']['score']['id']]['ref3']
    if type(raw_path) == pd.Series:
        raw_path = raw_path.iloc[0]
    d['mscz_path'] = raw_path
    test_ds[i] = d

In [23]:
id_train_ds = {d['d']['data']['score']['id']: d for d in train_ds}
with open('../data/train_metadata.json', 'w') as f:
    json.dump(id_train_ds, f, indent=2)
id_val_ds = {d['d']['data']['score']['id']: d for d in val_ds}
with open('../data/val_metadata.json', 'w') as f:
    json.dump(id_val_ds, f, indent=2)
id_test_ds = {d['d']['data']['score']['id']: d for d in test_ds}
with open('../data/test_metadata.json', 'w') as f:
    json.dump(id_test_ds, f, indent=2)

In [29]:
with open('../data/train_metadata.json', 'r') as f:
    id_train_ds = json.load(f)
with open('../data/val_metadata.json', 'r') as f:
    id_val_ds = json.load(f)
with open('../data/test_metadata.json', 'r') as f:
    id_test_ds = json.load(f)

In [24]:
os.system("mkdir /data2/zachary/musescore/coarse_data/train/")
os.system("mkdir /data2/zachary/musescore/coarse_data/val/")
os.system("mkdir /data2/zachary/musescore/coarse_data/test/")
for idx, d in id_train_ds.items():
    os.system(f"cp {d['mscz_path']} /data2/zachary/musescore/coarse_data/train/")
for idx, d in id_val_ds.items():
    os.system(f"cp {d['mscz_path']} /data2/zachary/musescore/coarse_data/val/")
for idx, d in id_test_ds.items():
    os.system(f"cp {d['mscz_path']} /data2/zachary/musescore/coarse_data/test/")

cp: error reading '/data2/zachary/musescore/data/a/G/QmaGjuqDxriJCdPX3XKELJT4iPU7MEzcuK72SE9oKdHhUH.mscz': Input/output error


In [25]:
for f in os.listdir('/data2/zachary/musescore/coarse_data/train/'):
    try:
        x = muspy.read_musescore(os.path.join('/data2/zachary/musescore/coarse_data/train/', f))
        if len(x.tracks[0]) < 10:
            os.system(f"rm /data2/zachary/musescore/coarse_data/train/{f}")
    except:
        os.system(f"rm /data2/zachary/musescore/coarse_data/train/{f}")
for f in os.listdir('/data2/zachary/musescore/coarse_data/val/'):
    try:
        x = muspy.read_musescore(os.path.join('/data2/zachary/musescore/coarse_data/val/', f))
        if len(x.tracks[0]) < 10:
            os.system(f"rm /data2/zachary/musescore/coarse_data/val/{f}")
    except:
        os.system(f"rm /data2/zachary/musescore/coarse_data/val/{f}")
for f in os.listdir('/data2/zachary/musescore/coarse_data/test/'):
    try:
        x = muspy.read_musescore(os.path.join('/data2/zachary/musescore/coarse_data/test/', f))
        if len(x.tracks[0]) < 10:
            os.system(f"rm /data2/zachary/musescore/coarse_data/test/{f}")
    except:
        os.system(f"rm /data2/zachary/musescore/coarse_data/test/{f}")

/home/zachary/miniconda3/envs/music/lib/python3.9/site-packages/muspy/inputs/musescore.py:962: MuseScoreWarning: Detected a legacy MuseScore version of 2.06. Data might not be loaded correctly.
  warnings.warn(
/home/zachary/miniconda3/envs/music/lib/python3.9/site-packages/muspy/inputs/musescore.py:962: MuseScoreWarning: Detected a legacy MuseScore version of 1.14. Data might not be loaded correctly.
  warnings.warn(


In [27]:
id_train_ds2 = {k:v for k,v in filter(lambda x: x[1]['mscz_path'].split("/")[-1] in os.listdir('/data2/zachary/musescore/coarse_data/train/'), id_train_ds.items())}

In [32]:
id_val_ds2 = {k:v for k,v in filter(lambda x: x[1]['mscz_path'].split("/")[-1] in os.listdir('/data2/zachary/musescore/coarse_data/val/'), id_val_ds.items())}


id_test_ds2 = {k:v for k,v in filter(lambda x: x[1]['mscz_path'].split("/")[-1] in os.listdir('/data2/zachary/musescore/coarse_data/test/'), id_test_ds.items())}

In [33]:

with open('../data/train_metadata.json', 'w') as f:
    json.dump(id_train_ds2, f, indent=2)
with open('../data/val_metadata.json', 'w') as f:
    json.dump(id_val_ds2, f, indent=2)
with open('../data/test_metadata.json', 'w') as f:
    json.dump(id_test_ds2, f, indent=2)

In [2]:
class LocalMuseScoreDataset(muspy.FolderDataset):

    _extension = 'mscz'
    _info = muspy.DatasetInfo("MuseScore Dataset")
    

    def read(self, filename) -> muspy.Music:
        """Read a file into a Music object."""
        obj = muspy.read_musescore(filename)
        return obj
    
    def to_pytorch_dataset(
        self,
        factory=None,
        representation: str = None,
        split_filename=None,
        splits=None,
        random_state=None,
        labpath=None,
        **kwargs):
        """Return the dataset as a PyTorch dataset.

        Parameters
        ----------
        factory : Callable, optional
            Function to be applied to the Music objects. The input is a
            Music object, and the output is an array or a tensor.
        representation : str, optional
            Target representation. See :func:`muspy.to_representation()`
            for available representation.
        split_filename : str or Path, optional
            If given and exists, path to the file to read the split
            from. If None or not exists, path to save the split.
        splits : float or list of float, optional
            Ratios for train-test-validation splits. If None, return the
            full dataset as a whole. If float, return train and test
            splits. If list of two floats, return train and test splits.
            If list of three floats, return train, test and validation
            splits.
        random_state : int, array_like or RandomState, optional
            Random state used to create the splits. If int or
            array_like, the value is passed to
            :class:`numpy.random.RandomState`, and the created
            RandomState object is used to create the splits. If
            RandomState, it will be used to create the splits.

        Returns
        -------
        :class:torch.utils.data.Dataset` or Dict of \
                :class:torch.utils.data.Dataset`
            Converted PyTorch dataset(s).

        """
        if representation is None and factory is None:
            raise TypeError(
                "One of `representation` and `factory` must be given."
            )
        if representation is not None and factory is not None:
            raise TypeError(
                "Only one of `representation` and `factory` can be given."
            )

        try:
            # pylint: disable=import-outside-toplevel
            from torch.utils.data import Dataset as TorchDataset
        except ImportError as err:
            raise ImportError("Optional package pytorch is required.") from err

        class TorchMusicFactoryDataset(TorchDataset):
            """A PyTorch dataset built from a Music dataset.

            Parameters
            ----------
            dataset : :class:`muspy.Dataset`
                Dataset object to base on.
            factory : Callable
                Function to be applied to the Music objects. The input is a
                Music object, and the output is an array or a tensor.

            """

            def __init__(
                self,
                dataset,
                factory,
                labpath,
                subset: str = "Full",
                indices=None,
            ):
                self.dataset = dataset
                self.factory = factory
                self.subset = subset
                self.indices = indices
                self.dataset.on_the_fly()
        
                with open(labpath, 'r') as f:
                    set_files = set(self.dataset._filenames)
                    self.metadata = json.load(f)
                    #self.metadata = {k: v for k,v in list(filter(lambda x: PosixPath(os.path.join(self.dataset.root, x[1]['mscz_path'].split("/")[-1])) in set_files, self.metadata.items()))}
                    self.metadata = {PosixPath(os.path.join(self.dataset.root, d['mscz_path'].split("/")[-1])): d for d in self.metadata.values()}
                    self.labels = {k: v['d']['data']['score']['complexity'] for k,v in self.metadata.items()}
                    self.labels = [x[1] for x in sorted(self.labels.items(), key=lambda y: self.dataset._filenames.index(y[0]))]
                    self.urls = {k: v['d']['data']['score']['share']['publicUrl'] for k,v in self.metadata.items()}
                    self.urls = [x[1] for x in sorted(self.urls.items(), key=lambda y: self.dataset._filenames.index(y[0]))]
                    self.names = {k: v['d']['data']['score']['share']['title'] for k,v in self.metadata.items()}
                    self.names = [x[1] for x in sorted(self.names.items(), key=lambda y: self.dataset._filenames.index(y[0]))]
                    self.metadata = [(x[0], x[1]) for x in sorted(self.metadata.items(), key=lambda y: self.dataset._filenames.index(y[0]))]
                self.dataset.use_converted()
                    
                if self.indices is not None:
                    self.indices = sorted(
                        idx for idx in self.indices if idx < len(self.dataset)
                    )

            def __repr__(self) -> str:
                return (
                    f"TorchMusicFactoryDataset(dataset={self.dataset}, "
                    f"factory={self.subset}, subset={self.factory})"
                )

            def __getitem__(self, index):
                if self.indices is None:
                    return self.factory(self.dataset[index]), self.labels[index]
                return self.factory(self.dataset[self.indices[index]]), self.labels[self.indices[index]]

            def __len__(self) -> int:
                if self.indices is None:
                    return len(self.dataset)
                return len(self.indices)

        class TorchRepresentationDataset(TorchMusicFactoryDataset):
            """A PyTorch music dataset.

            Parameters
            ----------
            dataset : :class:`muspy.Dataset`
                Dataset object to base on.
            representation : str
                Target representation. See
                :func:`muspy.to_representation()` for available
                representation.

            """

            def __init__(
                self,
                dataset,
                labpath,
                representation: str,
                subset="Full",
                indices=None,
                **kwargs,
            ):
                self.representation = representation

                def factory(music):
                    if representation == 'remi':
                        return to_remi_event_representation(music)
                    #return music.to_representation(representation, **kwargs)
                    return music

                super().__init__(
                    dataset, labpath=labpath, factory=factory, subset=subset, indices=indices
                )

            def __repr__(self) -> str:
                return (
                    f"TorchRepresentationDataset(dataset={self.dataset}, "
                    f"representation={self.representation}, "
                    f"subset={self.subset})"
                )

        # No split
        if splits is None:
            if representation is not None:
                return TorchRepresentationDataset(
                    self,labpath, representation, **kwargs
                )
            return TorchMusicFactoryDataset(self, factory, labpath=labpath)  # type: ignore

        datasets= {}
        indices_list = self.split(split_filename, splits, random_state)
        for key, value in indices_list.items():
            if representation is not None:
                datasets[key] = TorchRepresentationDataset(
                    self, representation, key, value, **kwargs
                )
            else:

                datasets[key] = TorchMusicFactoryDataset(
                    self, factory, key, value  # type: ignore
                )

        return datasets

In [3]:
train = LocalMuseScoreDataset('/data2/zachary/musescore/coarse_data/train/')
trainDataset = train.to_pytorch_dataset(representation='event', labpath='../data/train_metadata.json')
val = LocalMuseScoreDataset('/data2/zachary/musescore/coarse_data/val/')
valDataset = val.to_pytorch_dataset(representation='event', labpath='../data/val_metadata.json')
test = LocalMuseScoreDataset('/data2/zachary/musescore/coarse_data/test/')
testDataset = test.to_pytorch_dataset(representation='event', labpath='../data/test_metadata.json')

In [4]:
import pickle as pk

In [5]:
trainDataset[0]

(Music(metadata=Metadata(schema_version='0.2', source_filename='QmNLhKBea8ZAJB6Sxe1Wc7A6DHJ2tL5P9732Fhq1Gzm4XM.mscz', source_format='musescore'), resolution=480, tempos=[Tempo(time=0, qpm=76.00019999999999)], time_signatures=[TimeSignature(time=0, numerator=3, denominator=8), TimeSignature(time=5280, numerator=1, denominator=4), TimeSignature(time=5760, numerator=1, denominator=8), ...], barlines=[Barline(time=0), Barline(time=240), Barline(time=960), ...], beats=[Beat(time=0), Beat(time=240), Beat(time=720), ...], tracks=[Track(program=0, is_drum=False, name='Piano', notes=[Note(time=0, pitch=76, duration=120, velocity=64, pitch_str='E'), Note(time=120, pitch=75, duration=120, velocity=64, pitch_str='Eb'), Note(time=240, pitch=76, duration=120, velocity=64, pitch_str='E'), ...])]),
 2)

In [9]:
cntr = 0
for i, d in enumerate(trainDataset):
    try:
        d = (to_remi_event_representation(d[0]), d[1])
        with open(f"/data2/zachary/musescore/pk_small/train/{cntr}.pk", 'wb') as f:
            pk.dump(d, f)
        if cntr % 100 == 0:
            print(i)
        cntr += 1
    except:
        print(d[0].metadata.title)
cntr = 0
for i, d in enumerate(valDataset):
    try:
        d = (to_remi_event_representation(d[0]), d[1])
        with open(f"/data2/zachary/musescore/pk_small/val/{cntr}.pk", 'wb') as f:
            pk.dump(d, f)
        if cntr % 100 == 0:
            print(i)
        cntr += 1
    except:
        print(d[0].metadata.title)
cntr = 0
for i, d in enumerate(testDataset):
    try:
        d = (to_remi_event_representation(d[0]), d[1])
        with open(f"/data2/zachary/musescore/pk_small/test/{cntr}.pk", 'wb') as f:
            pk.dump(d, f)
        if cntr % 100 == 0:
            print(i)
        cntr += 1
    except:
        print(d[0].metadata.title)

0
GOYESCAS
Prelude in Dm
JUST DO IT
Prelude no. 2 in D minor
2 Konzertetüden
3am
CW DC TV Shows Mashup/Medley
Piece like Chopin/Mozart
Hasta que me olvides
Sonata 1
None
None
Sample 1
Arabesque
None
Lily 
None
BFG Division
Fado Serenata do Hylario
None
120
Kannana Kanne
Carnval Del Barrio
Rhapsodie Hongroise
Symphonia Claviensis (Page 1)
NUITS D'ÉTÉ A PAUSILIPPE
Chaconne in F
Set Sails - Espresso
None
Piano Sonata No. 1
Gaspard de la nuit
GLOOMY SUNDAY
Minijatura
Fuga
Nocturne No. 1
Pie Jesu
Guren No Yumiya
Smile Meditation
None
All Star (Dark and Dramatic version)
None
as
Piccolo solo stars and stripes
Un Sospiro
None
244
Stringere
"Inspirational"
Etude No. 10 in E Minor, Op. 15
Boogie Woogie Stomp
Full Disclosure (Steven Universe) [AmazingMIDI]
Quatre Mazurkas


/tmp/ipykernel_154468/3886845370.py:46: RuntimeWarning: invalid value encountered in long_scalars
  res * (time - measure_times[measure_idx]) / measure_length


None
None
Arabesque
None
Jeux d'eau
Dancing Line - The Christmas Eve
None
Foggy Night in D Major
Title
chimchar
360
Étude-Tableaux No. 8
Frogs Legs and Dragons Teeth
"Flying Mario"
Grievous Lady
CompExercise2
None
Fantaisie - Impromptu
Random Piece
None
Grande Fantaisie de 4 années
Sonatina for Pianoforte
None
472
Sonate
Piano Sonata No. 1
PUPA
Fairy Tail Main Theme (2014)
Prelude in A-flat Major
Prelude 2 in C minor
vocaloid songbook
579
RED ZONE
Title
None
Allegro con brio
Crimson skies
Op.212
Ambient
None
Sinking
SOME IMPOSSIBLE PIANO 
Concert Étude after Paganini's 3rd Caprice

Valzer Romantico
Douze Etudes VII
5 Preludes Op.1
Namae no Nai Kaibutsu
Barato Total
Alberti Bass Lincoln
Liebestraum No. 3 in A♭ Major
697
None
None
Nocturne
POTPOURRIS
I was bored
Prelude in B Minor
None
804
Nocturne in G minor op. no.3
None
Brightest Night
六月
Rhapsody no. 2
Andante_Allegro con Fuoco
The Glarf
Moonlight Sonata
None
Melodies of the Day
914
Danza de la moza donosa
Gaudeamus Igitur Paraphrase

In [8]:
with open("/data2/zachary/musescore/pk_data/test/0.pk", 'rb') as f:
    test_pk = pk.load(f)

In [20]:
len(torch.argwhere(to_remi_event_representation(test_pk[0]) == 0).flatten()) == len(test_pk[0].barlines)

True

In [27]:
test_pk[1][1]['d']['data']['score']['complexity']

2

In [ ]:
obj = muspy.read_musescore('/data2/zachary/musescore/coarse_data/train/QmNPiX2HMWvg8rBD6Wn4wQXW1T9TVGZbm256gHQ5X7gfEk.mscz')

In [ ]:
print([idx2tok[x] for x in to_remi_event_representation(obj.adjust_resolution(24)).numpy()])

In [6]:
from muspy.outputs.event import REMIEventSequence
from math import ceil
from operator import attrgetter, itemgetter
from typing import TYPE_CHECKING, Iterable, List, Tuple

import numpy as np
from bidict import bidict
from numpy import ndarray

def to_remi_event_sequence(music: "Music", res=24) -> REMIEventSequence:
    """Return a Music object as a REMIEventSequence object."""
    # Adjust resolution
    music.adjust_resolution(res)

    # Collect notes
    notes = []
    for track in music.tracks:
        notes.extend(track.notes)

    # Raise an error if no notes is found
    if not notes:
        raise RuntimeError("No notes found.")

    # Create a REMIEventSequence object
    seq = REMIEventSequence()
    seq.indexer = get_remi_indexer()

    # Collect measure times
    barline_times = [barline.time for barline in music.barlines]
    if barline_times[0] != 0:
        barline_times.insert(0, 0)
    measure_times = np.sort(barline_times)
    assert len(measure_times) > 1

    def _get_measure_position(time) -> Tuple[int, int]:
        measure_idx = np.searchsorted(measure_times, time, "right") - 1
        if measure_idx < len(measure_times) - 1:
            measure_length = (
                measure_times[measure_idx + 1] - measure_times[measure_idx]
            )
        else:
            measure_length = (
                measure_times[measure_idx] - measure_times[measure_idx - 1]
            )
        position = ceil(
            res * (time - measure_times[measure_idx]) / measure_length
        )
        return measure_idx, position % res

    # Collect events
    events: List[Tuple[Tuple[int, int], List[str]]] = []
    for barline_time in barline_times:
        events.append(
            (_get_measure_position(barline_time), [seq.to_bar_event()])
        )
    for tempo in music.tempos:
        events.append(
            (
                _get_measure_position(tempo.time),
                [seq.to_tempo_event(tempo.qpm)],
            )
        )
    for note in notes:
        events.append(
            (
                _get_measure_position(note.time),
                [
                    seq.to_note_on_event(note.pitch),
                    seq.to_note_velocity_event(note.velocity),
                    seq.to_note_duration_event(min(note.duration, res*2)),
                ],
            )
        )

    # Sort the events by time
    events.sort(key=itemgetter(0))

    # Append the events to the event sequence
    for i, event in enumerate(events):
        if event[1][0] != "bar":
            seq.append_event(seq.to_position_event(event[0][1]))
        seq.extend_events(event[1])

    seq.append_event('eos')
    return seq

def get_remi_indexer() -> bidict[str, int]:
    """Return the REMI indexer."""
    indexer = {}
    idx = 0
    # Beat event
    indexer["bar"] = idx
    idx += 1
    indexer["eos"] = idx
    idx += 1
    # Note-on events
    for i in range(128):
        indexer[f"note_on_{i}"] = idx
        idx += 1
    # Note-duration events
    for i in range(1, 65):
        indexer[f"note_duration_{i}"] = idx
        idx += 1
    # Note-velocity events
    for i in range(32):
        indexer[f"note_velocity_{i}"] = idx
        idx += 1
    # Position events
    for i in range(24):
        indexer[f"position_{i}"] = idx
        idx += 1
    # Tempo events
    for i in range(30, 210):
        indexer[f"tempo_{i}"] = idx
        idx += 1
    return bidict(indexer)

def to_remi_event_representation(music: "Music", dtype=int) -> ndarray:
    """Encode a Music object into the remi event representation."""
    seq = to_remi_event_sequence(music)
    return np.array(seq, dtype=dtype)

In [ ]:
get_remi_indexer()

In [ ]:
tok2idx = get_remi_indexer()
idx2tok = {v:k for k,v in tok2idx.items()}

In [ ]:
import pickle as pk

In [ ]:
with open("../MuseMorphose/pickles/msremi_vocab.pkl", 'wb') as f:
    pk.dump((tok2idx, idx2tok), f)

In [ ]:
train_dataloader = DataLoader(trainDataset)
val_dataloader = DataLoader(valDataset)
test_dataloader = DataLoader(testDataset)